# _utils.states

> TODO fill in description

In [ ]:
#| default_exp _utils.states

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio

import fbdev
from fbdev._utils.attr_container import AttrContainer
from fbdev._utils.events import ReadonlyEvent

In [ ]:
#|hide
show_doc(fbdev._utils.states.StateHandler)

---

### StateHandler

>      StateHandler (name, current_state, state_vals=[True, False])

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class StateHandler:
    def __init__(self, name, current_state, state_vals=[True, False]):
        self._name = name
        state_vals = list(state_vals) # Can be enums
        self._state_vals = state_vals
        if len(state_vals) != len(set(state_vals)): raise ValueError("`state_vals` must have all unique elements.")
        if current_state not in state_vals: raise ValueError("`current_state` must be in `state_vals`.")
        self._state_is_on = {state : asyncio.Event() for state in state_vals}
        self._state_is_on[current_state].set()
        self._state_is_off = {state : asyncio.Event() for state in state_vals}
        self._current_state = current_state
        for state in self._state_is_off:
            if state != current_state: self._state_is_off[state].set()
        
    @property
    def name(self) -> str: return self._name
        
    def check(self, state):
        return self._state_is_on[state].is_set()
    
    def get(self):
        return self._current_state
    
    def set(self, state):
        if state not in self._state_vals: raise ValueError(f"Invalid state: {state}. Possible states: {', '.join(self._state_vals)}")
        self._current_state = state
        for _state in self._state_is_on:
            if _state == state:
                self._state_is_on[_state].set()
                self._state_is_off[_state].clear()
            else:
                self._state_is_on[_state].clear()
                self._state_is_off[_state].set()
            
    def wait(self, state, target_value=True):
        if target_value: return self._state_is_on[state].wait()
        else: return self._state_is_off[state].wait()
        
    async def _event_func(self, state, event):
        await state.wait()
        event.set()
      
    def get_state_event(self, state, target_value=True):
        if target_value: return ReadonlyEvent(self._state_is_on[state])
        else: return ReadonlyEvent(self._state_is_off[state])
        
    def get_state_toggle_event(self, state, target_value=True):
        event = asyncio.Event()
        if target_value: asyncio.create_task(self._event_func(self._state_is_on[state], event))
        else: asyncio.create_task(self._event_func(self._state_is_off[state], event))
        return event
    
    def get_state_changed_event(self):
        return self.get_state_toggle_event(self._current_state, target_value=False)
    
    def __str__(self):
        return f"{self.name}: {self.get()}"
    
    def __repr__(self):
        return self.__str__()

In [ ]:
#|hide
show_doc(fbdev._utils.states.StateView)

---

### StateView

>      StateView (state_handler)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class StateView:
    def __init__(self, state_handler):
        self._state_handler: StateHandler = state_handler
        
    @property
    def name(self) -> str: return self._state_handler.name
        
    def check(self, state):
        return self._state_handler.check(state)
    
    def get(self):
        return self._state_handler._current_state
            
    def wait(self, state, state_value=True):
        return self._state_handler.wait(state, state_value)
      
    def get_state_event(self, state, state_value=True):
        return self._state_handler.get_state_event(state, state_value)
        
    def get_state_toggle_event(self, state, state_value=True):
        return self._state_handler.get_state_toggle_event(state, state_value)
    
    def __str__(self):
        return str(self._state_handler)
    
    def __repr__(self):
        return self.__str__()

In [ ]:
#|hide
show_doc(fbdev._utils.states.StateCollection)

---

### StateCollection

>      StateCollection ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class StateCollection(AttrContainer):
    def __init__(self) -> None:
        super().__init__({}, obj_name="StateCollection")
    
    def _add_state(self, state_handler: StateHandler, readonly=True):
        self._set(f"_{state_handler.name}", state_handler)
        if readonly:
            self._set(f"{state_handler.name}", StateView(state_handler))
        else:
            self._set(state_handler.name, state_handler)
            
    def __str__(self):
        return f'{self._obj_name}: {", ".join([f"{v}" for k,v in self._attrs.items() if not k.startswith("_")])}'